现基于iris数据集，使用 估计法，建立Sepal.Length、Sepal.Width、Petal.Length对Petal.Width的稳健回归模型

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import random
iris=pd.read_csv("http://image.cador.cn/data/iris.csv")
x = iris.iloc[:,[0,1,2]].values
y = iris.iloc[:,3].values

# 对x中的每个变量，随机替换5个较大值
row, col = x.shape
for k in range(col):
    x[random.sample(range(row),5),k] = np.random.uniform(10,30,5)

clf = linear_model.LinearRegression()
clf.fit(x,y)
x_input = np.c_[x,[1]*row]
col = col + 1
beta = list(clf.coef_) + [clf.intercept_]
c = 0.8
k = 0

In [2]:
# 迭代法求解
while k < 100:
    y0 = np.matmul(x_input,beta)
    epsilon = y - y0
    delta = np.std(epsilon)
    epsilon = epsilon/delta
    faik = [0]*col
    faikD = np.zeros((col,col))
    for i in range(row):
        if np.abs(epsilon[i]) <= c:
            xi = x_input[i,:]
            faik = faik + np.sin(epsilon[i]/c)*xi
            faikD = faikD - np.cos(epsilon[i]/c)*np.array([h*xi for h in xi])/(delta*c)
            
    b = np.matmul(np.linalg.inv(faikD),faik)
    beta = beta - b
    print(np.max(np.abs(b)))
    if np.max(np.abs(b)) < 1e-15:
        print("算法收敛，退出循环！")
        break
        
    k = k + 1

0.041583360442955575
0.13288028329048984
0.728241189475862
0.09522254097415245
0.0011120510881178504
1.7254532237701597e-07
6.460861534568558e-11
3.023389358104663e-14
6.178605827578578e-17
算法收敛，退出循环！


In [3]:
np.mean(np.abs(y - np.matmul(x_input,beta)))

0.4104814942387641

In [4]:
beta

array([ 0.00209922,  0.00210944,  0.41556885, -0.38645105])

现使用普通最小二乘法拟合回归系数，建立线性回归模型

In [5]:
clf.fit(x,y)
list(clf.coef_) + [clf.intercept_]

[0.09106041273347798,
 -0.006162626883046202,
 0.05853758944345057,
 0.3958411864670931]

In [6]:
np.mean(np.abs(clf.predict(x) - y))

0.5670366038984386